In [1]:
import shutil
from pathlib import Path

transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")
input_dir = Path("../deberta-v2-3-fast-tokenizer")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in [
    'tokenization_deberta_v2.py',
    'tokenization_deberta_v2_fast.py',
    "deberta__init__.py"]:
    if str(filename).startswith("deberta"):
        filepath = deberta_v2_path/str(filename).replace("deberta", "")
    else:
        filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)

In [2]:
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup,AdamW,AutoModel,AutoConfig
%env TOKENIZERS_PARALLELISM=true

from transformers.models.deberta_v2 import DebertaV2TokenizerFast

tokenizers.__version__: 0.11.0
transformers.__version__: 4.16.2


2022-03-26 08:07:44.480702: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


env: TOKENIZERS_PARALLELISM=true


In [3]:
# ========================================
# library
# ========================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_squared_error
%matplotlib inline
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.cuda.amp import autocast, GradScaler
import logging
from ast import literal_eval
import sys
from contextlib import contextmanager
import time
import random
from tqdm import tqdm
import os
import ast
import itertools
from sklearn.metrics import f1_score,recall_score
from torch.nn import Parameter
import torch.nn.functional as F

In [4]:
# ==================
# Constant
# ==================
FEATURES_PATH = "../data/features.csv"
PATIENT_NOTES_PATH = "../data/patient_notes.csv"
TRAIN_PATH = "../data/train.csv"
TEST_PATH = "../data/test.csv"
PRETRAIN_MODEL_PATH = "../output/pretrained/microsoft-deberta-v2-xlarge/microsoft-deberta-v2-xlarge-mlm-epoch-10.bin"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# ===============
# Configs
# ===============
ex = "032"
if not os.path.exists(f"../output/exp/ex{ex}"):
    os.makedirs(f"../output/exp/ex{ex}")
    os.makedirs(f"../output/exp/ex{ex}/ex{ex}_model")
    
OUTPUT_DIR = f"../output/exp/ex{ex}"
MODEL_PATH_BASE = f"../output/exp/ex{ex}/ex{ex}_model/ex{ex}"
LOGGER_PATH = f"../output/exp/ex{ex}/ex{ex}.txt"

SEED = 0
N_SPLITS = 5
SHUFFLE = True
num_workers = 4
BATCH_SIZE = 4
iters_to_accumulate = 1
n_epochs = 6
es_patience = 10
max_len = 512
weight_decay = 0.1
beta = (0.9, 0.98)
lr = 1e-5
num_warmup_steps_rate = 0.1
clip_grad_norm = 1.0

MODEL_PATH = "microsoft/deberta-v2-xlarge"
tokenizer = DebertaV2TokenizerFast.from_pretrained(MODEL_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#tokenizer.save_pretrained("../output/pretrained/microsoft-deberta-v3-large/mlm/")

In [7]:
# ===============
# Functions
# ===============
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def setup_logger(out_file=None, stderr=True, stderr_level=logging.INFO, file_level=logging.DEBUG):
    LOGGER.handlers = []
    LOGGER.setLevel(min(stderr_level, file_level))

    if stderr:
        handler = logging.StreamHandler(sys.stderr)
        handler.setFormatter(FORMATTER)
        handler.setLevel(stderr_level)
        LOGGER.addHandler(handler)

    if out_file is not None:
        handler = logging.FileHandler(out_file)
        handler.setFormatter(FORMATTER)
        handler.setLevel(file_level)
        LOGGER.addHandler(handler)

    LOGGER.info("logger set up")
    return LOGGER


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')
    
    
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2022-03-26 08:08:40,159 - INFO - logger set up


<RootLogger root (DEBUG)>

In [8]:
# ====================================================
# Main
# ====================================================
train = pd.read_csv(TRAIN_PATH)
train['annotation'] = train['annotation'].apply(ast.literal_eval)
train['location'] = train['location'].apply(ast.literal_eval)
features = pd.read_csv(FEATURES_PATH )
def preprocess_features(features):
    features.loc[27, 'feature_text'] = "Last-Pap-smear-1-year-ago"
    return features
features = preprocess_features(features)
patient_notes = pd.read_csv(PATIENT_NOTES_PATH)

In [9]:
train = train.merge(features, on=['feature_num', 'case_num'], how='left')
train = train.merge(patient_notes, on=['pn_num', 'case_num'], how='left')
display(train.head())

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...


In [10]:
# incorrect annotation
train.loc[338, 'annotation'] = ast.literal_eval('[["father heart attack"]]')
train.loc[338, 'location'] = ast.literal_eval('[["764 783"]]')

train.loc[621, 'annotation'] = ast.literal_eval('[["for the last 2-3 months"]]')
train.loc[621, 'location'] = ast.literal_eval('[["77 100"]]')

train.loc[655, 'annotation'] = ast.literal_eval('[["no heat intolerance"], ["no cold intolerance"]]')
train.loc[655, 'location'] = ast.literal_eval('[["285 292;301 312"], ["285 287;296 312"]]')

train.loc[1262, 'annotation'] = ast.literal_eval('[["mother thyroid problem"]]')
train.loc[1262, 'location'] = ast.literal_eval('[["551 557;565 580"]]')

train.loc[1265, 'annotation'] = ast.literal_eval('[[\'felt like he was going to "pass out"\']]')
train.loc[1265, 'location'] = ast.literal_eval('[["131 135;181 212"]]')

train.loc[1396, 'annotation'] = ast.literal_eval('[["stool , with no blood"]]')
train.loc[1396, 'location'] = ast.literal_eval('[["259 280"]]')

train.loc[1591, 'annotation'] = ast.literal_eval('[["diarrhoe non blooody"]]')
train.loc[1591, 'location'] = ast.literal_eval('[["176 184;201 212"]]')

train.loc[1615, 'annotation'] = ast.literal_eval('[["diarrhea for last 2-3 days"]]')
train.loc[1615, 'location'] = ast.literal_eval('[["249 257;271 288"]]')

train.loc[1664, 'annotation'] = ast.literal_eval('[["no vaginal discharge"]]')
train.loc[1664, 'location'] = ast.literal_eval('[["822 824;907 924"]]')

train.loc[1714, 'annotation'] = ast.literal_eval('[["started about 8-10 hours ago"]]')
train.loc[1714, 'location'] = ast.literal_eval('[["101 129"]]')

train.loc[1929, 'annotation'] = ast.literal_eval('[["no blood in the stool"]]')
train.loc[1929, 'location'] = ast.literal_eval('[["531 539;549 561"]]')

train.loc[2134, 'annotation'] = ast.literal_eval('[["last sexually active 9 months ago"]]')
train.loc[2134, 'location'] = ast.literal_eval('[["540 560;581 593"]]')

train.loc[2191, 'annotation'] = ast.literal_eval('[["right lower quadrant pain"]]')
train.loc[2191, 'location'] = ast.literal_eval('[["32 57"]]')

train.loc[2553, 'annotation'] = ast.literal_eval('[["diarrhoea no blood"]]')
train.loc[2553, 'location'] = ast.literal_eval('[["308 317;376 384"]]')

train.loc[3124, 'annotation'] = ast.literal_eval('[["sweating"]]')
train.loc[3124, 'location'] = ast.literal_eval('[["549 557"]]')

train.loc[3858, 'annotation'] = ast.literal_eval('[["previously as regular"], ["previously eveyr 28-29 days"], ["previously lasting 5 days"], ["previously regular flow"]]')
train.loc[3858, 'location'] = ast.literal_eval('[["102 123"], ["102 112;125 141"], ["102 112;143 157"], ["102 112;159 171"]]')

train.loc[4373, 'annotation'] = ast.literal_eval('[["for 2 months"]]')
train.loc[4373, 'location'] = ast.literal_eval('[["33 45"]]')

train.loc[4763, 'annotation'] = ast.literal_eval('[["35 year old"]]')
train.loc[4763, 'location'] = ast.literal_eval('[["5 16"]]')

train.loc[4782, 'annotation'] = ast.literal_eval('[["darker brown stools"]]')
train.loc[4782, 'location'] = ast.literal_eval('[["175 194"]]')

train.loc[4908, 'annotation'] = ast.literal_eval('[["uncle with peptic ulcer"]]')
train.loc[4908, 'location'] = ast.literal_eval('[["700 723"]]')

train.loc[6016, 'annotation'] = ast.literal_eval('[["difficulty falling asleep"]]')
train.loc[6016, 'location'] = ast.literal_eval('[["225 250"]]')

train.loc[6192, 'annotation'] = ast.literal_eval('[["helps to take care of aging mother and in-laws"]]')
train.loc[6192, 'location'] = ast.literal_eval('[["197 218;236 260"]]')

train.loc[6380, 'annotation'] = ast.literal_eval('[["No hair changes"], ["No skin changes"], ["No GI changes"], ["No palpitations"], ["No excessive sweating"]]')
train.loc[6380, 'location'] = ast.literal_eval('[["480 482;507 519"], ["480 482;499 503;512 519"], ["480 482;521 531"], ["480 482;533 545"], ["480 482;564 582"]]')

train.loc[6562, 'annotation'] = ast.literal_eval('[["stressed due to taking care of her mother"], ["stressed due to taking care of husbands parents"]]')
train.loc[6562, 'location'] = ast.literal_eval('[["290 320;327 337"], ["290 320;342 358"]]')

train.loc[6862, 'annotation'] = ast.literal_eval('[["stressor taking care of many sick family members"]]')
train.loc[6862, 'location'] = ast.literal_eval('[["288 296;324 363"]]')

train.loc[7022, 'annotation'] = ast.literal_eval('[["heart started racing and felt numbness for the 1st time in her finger tips"]]')
train.loc[7022, 'location'] = ast.literal_eval('[["108 182"]]')

train.loc[7422, 'annotation'] = ast.literal_eval('[["first started 5 yrs"]]')
train.loc[7422, 'location'] = ast.literal_eval('[["102 121"]]')

train.loc[8876, 'annotation'] = ast.literal_eval('[["No shortness of breath"]]')
train.loc[8876, 'location'] = ast.literal_eval('[["481 483;533 552"]]')

train.loc[9027, 'annotation'] = ast.literal_eval('[["recent URI"], ["nasal stuffines, rhinorrhea, for 3-4 days"]]')
train.loc[9027, 'location'] = ast.literal_eval('[["92 102"], ["123 164"]]')

train.loc[9938, 'annotation'] = ast.literal_eval('[["irregularity with her cycles"], ["heavier bleeding"], ["changes her pad every couple hours"]]')
train.loc[9938, 'location'] = ast.literal_eval('[["89 117"], ["122 138"], ["368 402"]]')

train.loc[9973, 'annotation'] = ast.literal_eval('[["gaining 10-15 lbs"]]')
train.loc[9973, 'location'] = ast.literal_eval('[["344 361"]]')

train.loc[10513, 'annotation'] = ast.literal_eval('[["weight gain"], ["gain of 10-16lbs"]]')
train.loc[10513, 'location'] = ast.literal_eval('[["600 611"], ["607 623"]]')

train.loc[11551, 'annotation'] = ast.literal_eval('[["seeing her son knows are not real"]]')
train.loc[11551, 'location'] = ast.literal_eval('[["386 400;443 461"]]')

train.loc[11677, 'annotation'] = ast.literal_eval('[["saw him once in the kitchen after he died"]]')
train.loc[11677, 'location'] = ast.literal_eval('[["160 201"]]')

train.loc[12124, 'annotation'] = ast.literal_eval('[["tried Ambien but it didnt work"]]')
train.loc[12124, 'location'] = ast.literal_eval('[["325 337;349 366"]]')

train.loc[12279, 'annotation'] = ast.literal_eval('[["heard what she described as a party later than evening these things did not actually happen"]]')
train.loc[12279, 'location'] = ast.literal_eval('[["405 459;488 524"]]')

train.loc[12289, 'annotation'] = ast.literal_eval('[["experienced seeing her son at the kitchen table these things did not actually happen"]]')
train.loc[12289, 'location'] = ast.literal_eval('[["353 400;488 524"]]')

train.loc[13238, 'annotation'] = ast.literal_eval('[["SCRACHY THROAT"], ["RUNNY NOSE"]]')
train.loc[13238, 'location'] = ast.literal_eval('[["293 307"], ["321 331"]]')

train.loc[13297, 'annotation'] = ast.literal_eval('[["without improvement when taking tylenol"], ["without improvement when taking ibuprofen"]]')
train.loc[13297, 'location'] = ast.literal_eval('[["182 221"], ["182 213;225 234"]]')

train.loc[13299, 'annotation'] = ast.literal_eval('[["yesterday"], ["yesterday"]]')
train.loc[13299, 'location'] = ast.literal_eval('[["79 88"], ["409 418"]]')

train.loc[13845, 'annotation'] = ast.literal_eval('[["headache global"], ["headache throughout her head"]]')
train.loc[13845, 'location'] = ast.literal_eval('[["86 94;230 236"], ["86 94;237 256"]]')

train.loc[14083, 'annotation'] = ast.literal_eval('[["headache generalized in her head"]]')
train.loc[14083, 'location'] = ast.literal_eval('[["56 64;156 179"]]')

In [11]:
train['annotation_length'] = train['annotation'].apply(len)

In [12]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=N_SPLITS)
groups = train['pn_num'].values
for n, (train_index, val_index) in enumerate(Fold.split(train, train['location'], groups)):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    2860
1    2860
2    2860
3    2860
4    2860
dtype: int64

In [13]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(text, feature_text,tokenizer):
    inputs =  tokenizer.encode_plus(text, feature_text, 
                           add_special_tokens=True,
                           max_length=max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    return inputs


def create_label(text, annotation_length, location_list,tokenizer):
    encoded = tokenizer.encode_plus(text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            return_offsets_mapping=True)
    offset_mapping = encoded['offset_mapping']
    ignore_idxes = np.where(np.array(encoded.sequence_ids()) != 0)[0]
    label = np.zeros(len(offset_mapping))
    label[ignore_idxes] = -1
    if annotation_length != 0:
        for location in location_list:
            for loc in [s.split() for s in location.split(';')]:
                start_idx = -1
                end_idx = -1
                start, end = int(loc[0]), int(loc[1])
                for idx in range(len(offset_mapping)):
                    if (start_idx == -1) & (start < offset_mapping[idx][0]):
                        start_idx = idx - 1
                    if (end_idx == -1) & (end <= offset_mapping[idx][1]):
                        end_idx = idx + 1
                if start_idx == -1:
                    start_idx = end_idx
                if (start_idx != -1) & (end_idx != -1):
                    label[start_idx:end_idx] = 1
    return label


class TrainDataset(Dataset):
    def __init__(self,df,tokenizer):
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.annotation_lengths = df['annotation_length'].values
        self.locations = df['location'].values
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.feature_texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.pn_historys[item], 
                               self.feature_texts[item],
                               self.tokenizer)
        label = create_label(self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item],
                             self.tokenizer)
        return {
              'token': torch.tensor(inputs["input_ids"], dtype=torch.long),
              'mask': torch.tensor(inputs["attention_mask"], dtype=torch.long),
              'token_type':torch.tensor(inputs["token_type_ids"], dtype=torch.long),
              "y":torch.tensor(label, dtype=torch.float32)
               }
    
class TrainDataset_aug(Dataset):
    def __init__(self,df,tokenizer,mask_aug_p,mask_ratio):
        self.feature_texts = df['feature_text'].values
        self.pn_historys = df['pn_history'].values
        self.annotation_lengths = df['annotation_length'].values
        self.locations = df['location'].values
        self.tokenizer = tokenizer
        self.mask_aug_p = mask_aug_p
        self.mask_ratio = mask_ratio

    def __len__(self):
        return len(self.feature_texts)
    
    def mask_augment(self, inputs):
        all_inds = np.arange(1, len(inputs["input_ids"]) - 1)
        n_mask = max(int(len(all_inds) * self.mask_ratio), 1)
        np.random.shuffle(all_inds)
        mask_inds = all_inds[:n_mask]
        sep_ind = np.where(np.array(inputs["input_ids"]) == 2)[0]
        mask_inds = np.array([i for i in mask_inds if i < sep_ind[0]])
        inputs_ids = np.array(inputs["input_ids"])
        inputs_ids[mask_inds] = tokenizer.mask_token_id
        inputs["input_ids"] = list(inputs_ids)
        return inputs
    
    def __getitem__(self, item):
        inputs = prepare_input(self.pn_historys[item], 
                               self.feature_texts[item],
                               self.tokenizer)
        label = create_label(self.pn_historys[item], 
                             self.annotation_lengths[item], 
                             self.locations[item],
                             self.tokenizer)
        if float(torch.rand(1)) < self.mask_aug_p:
            inputs = self.mask_augment(inputs)
        return {
              'token': torch.tensor(inputs["input_ids"], dtype=torch.long),
              'mask': torch.tensor(inputs["attention_mask"], dtype=torch.long),
              'token_type':torch.tensor(inputs["token_type_ids"], dtype=torch.long),
              "y":torch.tensor(label, dtype=torch.float32)
               }

In [14]:
def create_labels_for_scoring(df):
    # example: ['0 1', '3 4'] -> ['0 1; 3 4']
    df['location_for_create_labels'] = [ast.literal_eval(f'[]')] * len(df)
    for i in range(len(df)):
        lst = df.loc[i, 'location']
        if lst:
            new_lst = ';'.join(lst)
            df.loc[i, 'location_for_create_labels'] = ast.literal_eval(f'[["{new_lst}"]]')
    # create labels
    truths = []
    for location_list in df['location_for_create_labels'].values:
        truth = []
        if len(location_list) > 0:
            location = location_list[0]
            for loc in [s.split() for s in location.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                truth.append([start, end])
        truths.append(truth)
    return truths


def get_char_probs(texts, predictions, tokenizer):
    results = [np.zeros(len(t)) for t in texts]
    for i, (text, prediction) in enumerate(zip(texts, predictions)):
        encoded = tokenizer(text, 
                            add_special_tokens=True,
                            return_offsets_mapping=True)
        for idx, (offset_mapping, pred) in enumerate(zip(encoded['offset_mapping'], prediction)):
            start = offset_mapping[0]
            end = offset_mapping[1]
            results[i][start:end] = pred
    return results


def get_results(char_probs, th=0.5):
    results = []
    for char_prob in char_probs:
        result = np.where(char_prob >= th)[0] + 1
        if len(result) > 0:
            if result[0] == 1:
                result = np.concatenate([np.array([0]),result],axis=0)
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result = [f"{min(r)} {max(r)}" for r in result]
        result = ";".join(result)
        results.append(result)
    return results

def get_results_pp(char_probs,txts,th=0.5,case=0):
    results = []
    for char_prob,txt in zip(char_probs,txts):
        result = np.where(char_prob >= th)[0] + 1
        if len(result) > 0:
            if result[0] == 1:
                result = np.concatenate([np.array([0]),result],axis=0)
        result = [list(g) for _, g in itertools.groupby(result, key=lambda n, c=itertools.count(): n - next(c))]
        result_ = []
        if case != 9:
            for r in result:
                if r[0] >= 1:
                    if txt[r[0] - 1] != ' ':
                        result_.append([r[0]-1] + r)
                    else:
                        result_.append(r)
                else:
                    result_.append(r)
            result_ = [f"{min(r)} {max(r)}" for r in result_]
            result_ = ";".join(result_)
            results.append(result_)
        else:
            result = [f"{min(r)} {max(r)}" for r in result]
            result = ";".join(result)
            results.append(result)
    return results


def get_predictions(results):
    predictions = []
    for result in results:
        prediction = []
        if result != "":
            for loc in [s.split() for s in result.split(';')]:
                start, end = int(loc[0]), int(loc[1])
                prediction.append([start, end])
        predictions.append(prediction)
    return predictions

def get_score(y_true, y_pred):
    score,re_score = span_micro_f1(y_true, y_pred)
    return score,re_score

def span_micro_f1(truths,preds):
    """
    Micro f1 on spans.

    Args:
        preds (list of lists of two ints): Prediction spans.
        truths (list of lists of two ints): Ground truth spans.

    Returns:
        float: f1 score.
    """
    bin_preds = []
    bin_truths = []
    for pred, truth in zip(preds, truths):
        if not len(pred) and not len(truth):
            continue
        length = max(np.max(pred) if len(pred) else 0, np.max(truth) if len(truth) else 0)
        bin_preds.append(spans_to_binary(pred, length))
        bin_truths.append(spans_to_binary(truth, length))
    return micro_f1(bin_preds, bin_truths)

def spans_to_binary(spans, length=None):
    """
    Converts spans to a binary array indicating whether each character is in the span.

    Args:
        spans (list of lists of two ints): Spans.

    Returns:
        np array [length]: Binarized spans.
    """
    length = np.max(spans) if length is None else length
    binary = np.zeros(length)
    for start, end in spans:
        binary[start:end] = 1
    return binary

def micro_f1(preds, truths):
    """
    Micro f1 on binary arrays.

    Args:
        preds (list of lists of ints): Predictions.
        truths (list of lists of ints): Ground truths.

    Returns:
        float: f1 score.
    """
    # Micro : aggregating over all instances
    preds = np.concatenate(preds)
    truths = np.concatenate(truths)
    return f1_score(truths, preds), recall_score(truths, preds)

In [15]:
class custom_model(nn.Module):
    def __init__(self):
        super(custom_model, self).__init__()
        self.model = AutoModel.from_pretrained(
            MODEL_PATH, 
        )
        self.config = AutoConfig.from_pretrained(MODEL_PATH)
        self.dropout1 = nn.Dropout(p=0.2)
        self.ln1 = nn.LayerNorm(1536)
        self.linear1 = nn.Linear(1536,768)
        self.ln2 = nn.LayerNorm(768)
        self.relu = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.2)
        self.linear2 = nn.Linear(768,1)
        self._init_weights(self.linear1)
        self._init_weights(self.linear2)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
            
    def forward(self, ids, mask,token_type):
        # pooler
        emb = self.model(ids, mask,token_type)["last_hidden_state"]
        output = self.ln1(emb)
        output = self.dropout1(output)
        output = self.linear1(output)
        output = self.ln2(output)
        output = self.relu(output)
        output = self.dropout2(output)
        output = self.linear2(output)
        return output
    
class FocalLoss(nn.Module):
    def __init__(self, reduction='none', alpha=1, gamma=2):
        super().__init__()
        self.reduction = reduction
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        bce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-bce_loss)
        loss = self.alpha * (1. - pt)**self.gamma * bce_loss
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss


class SmoothFocalLoss(nn.Module):
    def __init__(self, reduction='none', alpha=1, gamma=2, smoothing=0.0):
        super().__init__()
        self.reduction = reduction
        self.focal_loss = FocalLoss(reduction='none', alpha=alpha, gamma=gamma)
        self.smoothing = smoothing

    @staticmethod
    def _smooth(targets:torch.Tensor, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothFocalLoss._smooth(targets, self.smoothing)
        loss = self.focal_loss(inputs, targets)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss
    
def collate(d,train=True):
    mask_len = int(d["mask"].sum(axis=1).max())
    if train:
        return {"token" : d['token'][:,:mask_len],
                 "mask" : d['mask'][:,:mask_len],
                 "y" : d['y'][:,:mask_len],
                 "token_type" : d['token_type'][:,:mask_len],
                  "max_len" : mask_len}
    else:
        return {"token" : d['token'][:,:mask_len],
                 "mask" : d['mask'][:,:mask_len],
                 "token_type" : d['token_type'][:,:mask_len],
                  "max_len" : mask_len}

In [16]:
train

,id,case_num,pn_num,feature_num,annotation,location,feature_text,pn_history,annotation_length,fold
0,00016_000,0,16,0,[dad with recent heart attcak],[696 724],Family-history-of-MI-OR-Family-history-of-myoc...,HPI: 17yo M presents with palpitations. Patien...,1,4
1,00016_001,0,16,1,"[mom with ""thyroid disease]",[668 693],Family-history-of-thyroid-disorder,HPI: 17yo M presents with palpitations. Patien...,1,4
2,00016_002,0,16,2,[chest pressure],[203 217],Chest-pressure,HPI: 17yo M presents with palpitations. Patien...,1,4
3,00016_003,0,16,3,"[intermittent episodes, episode]","[70 91, 176 183]",Intermittent-symptoms,HPI: 17yo M presents with palpitations. Patien...,2,4
4,00016_004,0,16,4,[felt as if he were going to pass out],[222 258],Lightheaded,HPI: 17yo M presents with palpitations. Patien...,1,4
...,...,...,...,...,...,...,...,...,...,...
14295,95333_912,9,95333,912,[],[],Family-history-of-migraines,Stephanie madden is a 20 year old woman compla...,0,4
14296,95333_913,9,95333,913,[],[],Female,Stephanie madden is a 20 year old woman compla...,0,4
14297,95333_914,9,95333,914,[photobia],[274 282],Photophobia,Stephanie madden is a 20 year old woman compla...,1,4
14298,95333_915,9,95333,915,[no sick contacts],[421 437],No-known-illness-contacts,Stephanie madden is a 20 year old woman compla...,1,4


In [ ]:
# ================================
# train
# ================================
with timer("deberta_v2_xlarge"):
    set_seed(SEED)
    oof_pred = np.ndarray((len(train),max_len))
    for fold in range(5):
        print(f"===============fold{fold}:start===============")
        x_train  = train.loc[train["fold"] != fold].reset_index(drop=True)
        x_val =  train.loc[train["fold"] == fold].reset_index(drop=True)
        valid_texts = x_val['pn_history'].values
        valid_labels = create_labels_for_scoring(x_val)
        
        # dataset
        train_ = TrainDataset(x_train,tokenizer)
        train_aug = TrainDataset_aug(x_train,tokenizer,0.5,0.15)
        val_ = TrainDataset(x_val,tokenizer)
        
        # loader
        train_loader = DataLoader(dataset=train_aug, batch_size=BATCH_SIZE, shuffle = True ,pin_memory=True)
        #train_aug_loader = DataLoader(dataset=train_aug, batch_size=BATCH_SIZE, shuffle = True ,pin_memory=True)
        val_loader = DataLoader(dataset=val_, batch_size=BATCH_SIZE, shuffle = False , pin_memory=True)
        
        # model
        model = custom_model()
        model.load_state_dict(torch.load(PRETRAIN_MODEL_PATH),strict=False)
        model = model.to(device)
        
        # optimizer, scheduler
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          lr=lr,
                          betas=beta,
                          weight_decay=weight_decay,
                          )
        num_train_optimization_steps = int(len(train_loader) * n_epochs)
        num_warmup_steps = int(num_train_optimization_steps * num_warmup_steps_rate)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=num_warmup_steps,
                                                    num_training_steps=num_train_optimization_steps)
        
        criterion = SmoothFocalLoss(reduction="mean", alpha=1, gamma=2, smoothing=0.0001)
        best_val = 0
        
        for epoch in range(n_epochs):
            print(f"============start epoch:{epoch}============")
            model.train() 
            val_losses_batch = []
            scaler = GradScaler()
            for i, d in tqdm(enumerate(train_loader),total=len(train_loader)):
                d = collate(d)
                ids = d['token'].to(device)
                mask = d['mask'].to(device)
                token_type = d["token_type"].to(device)
                labels = d['y'].to(device)
                labels = labels.unsqueeze(-1)
                #labels = labels.unsqueeze(-1)
                #optimizer.zero_grad()
                with autocast():
                    output = model(ids,mask,token_type)
                    loss = criterion(output[labels != -1], labels[labels != -1])
                    loss = loss / iters_to_accumulate
                scaler.scale(loss).backward()
                if (i + 1) % iters_to_accumulate == 0:
                    # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                #torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
                #scaler.step(optimizer)
                #scaler.update()
                scheduler.step()
            
            val_preds = np.ndarray((0,max_len,1))
            model.eval()  # switch model to the evaluation mode
            with torch.no_grad():  
                for d in tqdm(val_loader,total=len(val_loader)):
                    # =========================
                    # data loader
                    # =========================
                    d = collate(d)
                    ids = d['token'].to(device)
                    mask = d['mask'].to(device)
                    token_type = d["token_type"].to(device)
                    with autocast():
                        outputs = model(ids, mask,token_type)
                    outputs = np.concatenate([outputs.sigmoid().detach().cpu().numpy(),np.zeros([len(outputs),max_len - d["max_len"],1])],axis=1)
                    val_preds = np.concatenate([val_preds, outputs], axis=0)
            char_probs = get_char_probs(valid_texts, val_preds,  tokenizer)
            results = get_results(char_probs,th=0.5)
            preds = get_predictions(results)
            score,re_score = get_score(valid_labels, preds)
            LOGGER.info(f'{fold},{epoch}:val_score:{score}')
            if best_val < score:
                print("save model weight")
                best_val = score
                best_val_preds = val_preds
                torch.save(model.state_dict(), MODEL_PATH_BASE + f"_{fold}.pth") # Saving current best model
        oof_pred[train["fold"] == fold,:] = best_val_preds.reshape([-1,max_len])
        np.save(OUTPUT_DIR + f"/ex{ex}_oof_npy_{fold}.npy",best_val_preds)
    # cv
    valid_texts = train['pn_history'].values
    valid_labels = create_labels_for_scoring(train)
    char_probs = get_char_probs(valid_texts, oof_pred,  tokenizer)
    results = get_results(char_probs, th=0.5)
    preds = get_predictions(results)
    score = get_score(valid_labels, preds)
    LOGGER.info(f'cv:{score}')
    np.save(OUTPUT_DIR + f"/ex{ex}_oof.npy",oof_pred)